# GSM8K with Langfun's Evaluation Framework

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/google/langfun/blob/main/docs/notebooks/langfun_eval.ipynb)

This notebook demonstrates how to build LLM benchmarks using Langfun's evaluation framework. By the end of this notebook, you will learn:

- How to create a new benchmark.
- How to execute a benchmark.
- How to view benchmark details.

We recommend opening this notebook in Colab, as GitHub's file preview does not render generated HTML views correctly.

In [ ]:
!pip install langfun --pre

In [ ]:
from typing import Any

import langfun as lf
import pandas as pd
import pyglove as pg

# Evaluation Input Functor
# NOTE: pg.functor decorator generates `pg.Object` subclass with a `__call__`
# method. It is commonly used for early or partial binding.
@pg.functor()
def gsm8k_inputs(num_examples=10):
  splits = {'train': 'main/train-00000-of-00001.parquet', 'test': 'main/test-00000-of-00001.parquet'}
  df = pd.read_parquet("hf://datasets/openai/gsm8k/" + splits["test"])
  return [
      pg.Dict(
          question=df['question'][i],
          answer=int(df['answer'][i].split('#### ')[1]),
      ) for i in range(num_examples)
  ]


#
# Output schema for lf.query.
#
class Answer(pg.Object):
  step_by_step_thoughts: list[str]
  final_answer: int


#
# Example of an evaluation class.
#
class Gsm8k(lf.eval.v2.Evaluation):
  """Gsm8K evaluation."""

  # `inputs` is a field declared from `lf.eval.v2.Evaluation`.
  inputs = gsm8k_inputs(num_examples=2)

  # User argument.
  lm: lf.LanguageModel

  #
  # Definition of the metric class.
  #
  class Match(lf.eval.v2.metrics.Match):

    def match(self, example_input: pg.Dict, output: Answer) -> bool:
      return example_input.answer == output.final_answer

  # Specification for which metrics to use.
  metrics = [Match()]

  # The processing logic, which maps an item from input functor to an output
  # which will be used for metric computation.
  def process(
      self,
      example: lf.eval.v2.Example
  ) -> tuple[
      Answer,              # Output for metric to process.
      dict[str, Any]       # Metadata for display.
  ]:
    with lf.track_queries() as queries:
      answer = lf.query(
          example.input.question,
          Answer,
          lm=self.lm
      )
    return answer, dict(queries=queries)


In [ ]:
import os

os.environ['OPENAI_API_KEY'] = '<Update this field with your OpenAI key>'

benchmark = Gsm8k(
    # Use `pg.oneof` to enable benchmark sweeping, which runs multiple evaluations
    # in parallel.
    lm=pg.oneof([
        lf.llms.Gpt4oMini(),
        lf.llms.GptO1Mini()
    ])
)
root_dir = '/tmp/test_run'
run = benchmark.run(root_dir, 'new')

Gsm8k(progress=Progress(num_total=None, num_processed=0, num_failed=0, num_skipped=0, start_time=None, stop_time=None, execution_summary=TimeIt.StatusSummary(breakdown={})), usage_summary=UsageSummary(cached=AggregatedUsage(total=LMSamplingUsage(prompt_tokens=0, completion_tokens=0, total_tokens=0, num_requests=0, estimated_cost=0.0, retry_stats=RetryStats(num_occurences=0, total_wait_interval=0.0, total_call_interval=0.0, errors={})), breakdown={}), uncached=AggregatedUsage(total=LMSamplingUsage(prompt_tokens=0, completion_tokens=0, total_tokens=0, num_requests=0, estimated_cost=0.0, retry_stats=RetryStats(num_occurences=0, total_wait_interval=0.0, total_call_interval=0.0, errors={})), breakdown={})), plugins=[], inputs=gsm8k_inputs(num_examples=2), metrics=[0: Match(name='match', oop_errors=Rate(data_points=[], total=0), non_oop_errors=Rate(data_points=[], total=0), matches=Rate(data_points=[], total=0), mismatches=Rate(data_points=[], total=0))], max_workers=32, lm=OneOf(name=None, hints=None, num_choices=1, candidates=[0: Gpt4oMini(sampling_options=LMSamplingOptions(temperature=None, max_tokens=None, n=1, top_k=40, top_p=None, stop=None, random_seed=None, logprobs=False, top_logprobs=None), cache=None, max_concurrency=None, timeout=120.0, max_attempts=5, retry_interval=(5, 60), exponential_backoff=True, max_retry_interval=300, debug=False, api_endpoint='https://api.openai.com/v1/chat/completions', model='gpt-4o-mini', headers=None, multimodal=True, api_key=None, organization=None, project=None), 1: GptO1Mini(sampling_options=LMSamplingOptions(temperature=None, max_tokens=None, n=1, top_k=40, top_p=None, stop=None, random_seed=None, logprobs=False, top_logprobs=None), cache=None, max_concurrency=None, timeout=None, max_attempts=5, retry_interval=(5, 60), exponential_backoff=True, max_retry_interval=300, debug=False, api_endpoint='https://api.openai.com/v1/chat/completions', model='o1-mini', headers=None, multimodal=False, api_key=None, organization=None, project=None)], choices_distinct=True, choices_sorted=False, where=None))

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Study the evaluation details of a dataset

In [ ]:
pg.view(
    benchmark.leaf_nodes[0],
    extra_flags=dict(
        current_run=run,
        interactive=False,
        card_view=False,
    ),
)

DefinitionMetric: matchLogs
matchesmismatchesoop_errorsnon_oop_errors
21


# View an detailed example

In [ ]:
example_id = 1  # @param int
pg.view(benchmark.leaf_nodes[0].state.evaluated_examples[example_id])

Html(content='<div class="eval-example"><div><div><span class="label example-id">#1</span><a class="label next" href="2.html">▶</a></div><div class="example-container"><div class="label-container"><span class="label usage-summary badge" id="control-138085545196752">0.000</span><span class="tooltip" id="control-138085545128208">UsageSummary(\n  cached = AggregatedUsage(\n    total = LMSamplingUsage(\n      prompt_tokens = 0,\n      completion_tokens = 0,\n      total_tokens = 0,\n      num_requests = 0,\n      estimated_cost = 0.000,\n      retry_stats = RetryStats(\n        num_occurences = 0,\n        total_wait_interval = 0.000,\n        total_call_interval = 0.000,\n        errors = {}\n      )\n    ),\n    breakdown = {}\n  ),\n  uncached = AggregatedUsage(\n    total = LMSamplingUsage(\n      prompt_tokens = 180,\n      completion_tokens = 136,\n      total_tokens = 316,\n      num_requests = 1,\n      estimated_cost = 0.000,\n      retry_stats = RetryStats(\n        num_occurences = 0,\n        total_wait_interval = 0.000,\n        total_call_interval = 2.882,\n        errors = {}\n      )\n    ),\n    breakdown = {\n      OpenAI(gpt-4o-mini) = LMSamplingUsage(\n        prompt_tokens = 180,\n        completion_tokens = 136,\n        total_tokens = 316,\n        num_requests = 1,\n        estimated_cost = 0.000,\n        retry_stats = RetryStats(\n          num_occurences = 0,\n          total_wait_interval = 0.000,\n          total_call_interval = 2.882,\n          errors = {}\n        )\n      )\n    }\n  )\n)</span></div><div class="label-group"><span class="label match badge group-value">match</span></div></div></div><table class="tab-control"><tr><td><div class="tab-button-group top" id="control-138085542756816-button-group"><button class="tab-button" onclick="openTab(event, \'control-138085542756816\', \'control-138085542756816-0\')"><span class="label">Input</span></button><button class="tab-button" onclick="openTab(event, \'control-138085542756816\', \'control-138085542756816-1\')"><span class="label">Output</span></button><button class="tab-button selected" onclick="openTab(event, \'control-138085542756816\', \'control-138085542756816-2\')"><span class="label">Output Metadata</span></button></div></td></tr><tr><td><div class="tab-content-group top" id="control-138085542756816-content-group"><div class="tab-content" id="control-138085542756816-0"><details open class="pyglove dict"><summary><div class="summary-title">Dict(...)</div><span class="tooltip">{\n  &#x27;question&#x27;: &quot;Janet’s ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. She sells the remainder at the farmers&#x27; market daily for $2 per fresh duck egg. How much in dollars does she make every day...&quot;,\n  &#x27;answer&#x27;: 18\n}</span></summary><div class="complex-value dict"><details open class="pyglove str"><summary><div class="summary-name">question<span class="tooltip">input.question</span></div><div class="summary-title">str</div><span class="tooltip">&quot;Janet’s ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. She sells the remainder at the farmers&#x27; market daily for $2 per fresh duck egg. How much in dollars does she make every day...&quot;</span></summary><span class="simple-value str">Janet’s ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. She sells the remainder at the farmers&#x27; market daily for $2 per fresh duck egg. How much in dollars does she make every day at the farmers&#x27; market?</span></details><details open class="pyglove int"><summary><div class="summary-name">answer<span class="tooltip">input.answer</span></div><div class="summary-title">int</div><span class="tooltip">18</span></summary><span class="simple-value int">18</span></details></div></details></div><div class="tab-conten